In [1]:
import os
import numpy as np

import torch
import pandas as pd
import whisper
import torchaudio
from whisper.audio import (
    log_mel_spectrogram,
    pad_or_trim,
)
from tqdm import tqdm
from jiwer import wer
from data import *
from suta import *

from omegaconf import OmegaConf
args = OmegaConf.load("config.yaml")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
dataset = load_dataset(['test-other'], 'librispeech', 'LibriSpeech', 1, extra_noise=0.01)

Read text: 100%|██████████| 2939/2939 [00:00<00:00, 76211.04it/s]

[INFO]    There are 2939 samples.


In [3]:
model = whisper.load_model("tiny")

In [4]:
options = whisper.DecodingOptions(language="en", without_timestamps=True, temperature=0)

In [21]:
hypotheses = []
references = []
from whisper.normalizers import EnglishTextNormalizer

normalizer = EnglishTextNormalizer()
count = 0
for batch in tqdm(dataset):
    count+=1
    if count > 1:
        break
    lens, wavs, texts, files = batch
    # the code is for batch size = 1
    if args.asr == 'large' or args.asr == 'large_v2' or args.asr == 'large_v3':
        mel = log_mel_spectrogram(pad_or_trim(wavs[0]), n_mels=128).unsqueeze(0).to(DEVICE)
    else:
        mel = log_mel_spectrogram(pad_or_trim(wavs[0])).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        results = model.decode(mel, options)
    print([result.text for result in results])
    print(wer(normalizer(texts[0]), normalizer(results[0].text)))

  0%|          | 1/2939 [00:00<19:21,  2.53it/s]

['I am willing to enter into competition with the agents and feel able to surpass them. For some sense, early days in which I made the medals of Pope Clemens, I have learned so much that I can now produce far better pieces of the kind. I think I can also outdo the coins I struck for Duke Alexander, which are still held in high esteem. In life manner, I could make the new large pieces of gold and silver plates. As I did so often for that noble monor, two Francis of France. Thanks to the great conveniences he allowed me, without ever losing time.']
0.20168067226890757
